# Run a simple flow using just the SDP directly

This notebook is based on the `notebooks/jupyter_notebook_simple_execution.ipynb` developed by the Naledi team. In the future, it may be prudent to modify that notebook so that it can easily run in the Mid ITF.

This notebook assumes that there is an instance of the SDP running in the cluster. The SDP is deployed along with the SUT. See the following gitlab jobs:
* deploy-sut-on-demand 
* deploy-integration

# 1 Import required libraries and create helpers

In [ ]:
import sys

sys.path.append("../../src")

import json
import logging
import os
import random
from datetime import date
from time import sleep, time

import ska_ser_logging
from ska_sdp_cbf_emulator import transmitters
from ska_sdp_cbf_emulator.data_source import MeasurementSetDataSourceConfig
from ska_sdp_cbf_emulator.packetiser import SenderConfig, packetise
from tango import DeviceProxy

from ska_mid_jupyter_scripting.cluster.configure import set_cluster


def wait_for_state(device, desired_state):
    print(f"State: {device.obsState.name}")
    while device.obsState != desired_state:
        print(f"State: {device.obsState.name}")
        sleep(0.5)
    print(f"State: {device.obsState.name}")

# 2 Setup base variables

This section is what is used to setup the remaining sections of code, so update this as required.

This section will generate random IDs, so it is possible that it might cause a conflict later when resourcing in section 7. To resolve the issue, run this section again, and then run section 7.

In [ ]:
MS_INPUT_NAME = "../../data/AA05LOW.ms"
namespace = "integration"  # Edit this line to have your namespace name
subarray_id = "01"  # Edit this line to select the subarray
enable_logging = True
use_network_attachment_definition = True

today = date.today().strftime("%Y%m%d")
random_id = random.randint(0, 99999)

EXECUTION_BLOCK_ID = f"eb-test-{today}-{random_id:05d}"
PROCESSING_BLOCK_ID_REALTIME = f"pb-testrealtime-{today}-{random_id:05d}"
KAFKA_HOST = f"ska-sdp-kafka.{namespace}.svc:9092"

set_cluster(
    namespace=namespace,
    database_name="tango-databaseds",
    polling=True,
    facility="itf",
)

if enable_logging:
    ska_ser_logging.configure_logging(logging.DEBUG)

# 3 Download a sample data file

You can use any valid .MS file here, however make sure that the extraction extracts only a single level deep.

There is an example file already in this repo, so this section can be skipped.

Other examples of data files can be found [here](https://gitlab.com/ska-telescope/sdp/ska-sdp-realtime-receive-core/-/tree/main/data?ref_type=heads)

In [ ]:
if not os.path.isdir(MS_INPUT_NAME):
    !curl https://gitlab.com/ska-telescope/sdp/ska-sdp-realtime-receive-core/-/raw/main/data/AA05LOW.ms.tar.gz --output AA05LOW.ms.tar.gz
    !tar -xzf AA05LOW.ms.tar.gz

# 4 Open Helper Dashboards

This is a link to the Signal Display. On which you will find links to the Grafana dashboard and Kibana logs.

Documentation for the Signal Displays can be found [here](https://developer.skao.int/projects/ska-sdp-qa-display/en/latest/)

In [ ]:
print(f"https://k8s.miditf.internal.skao.int/{namespace}/signal/display/")

# 5 Connect to the SubArray and Turn on

If the SubArray is off this will turn it on, otherwise it will stay on.

In [ ]:
d = DeviceProxy(f"mid-sdp/subarray/{subarray_id}")
d.state()
if d.state().name == "OFF":
    d.On()
d.state()
wait_for_state(d, 0)  # EMPTY

# 6 Create Assign Resources

This config includes the setup to include:
* The Signal Displays as per [this test](https://gitlab.com/ska-telescope/sdp/ska-sdp-integration/-/blob/master/tests/integration/test_qa_metric_receive.py?ref_type=heads).
* The MS writer processor. If you do not want to write a product to disk, remove `"mswriter",` from the `processors` dictionary.

Documentation links:
* The config is based on the LOW setup which can be found in the [SDP Integrations repo](https://gitlab.com/ska-telescope/sdp/ska-sdp-integration/-/blob/master/tests/resources/subarray-json/low.json?ref_type=heads) (LOW is used here, as the sample data by default is a LOW object)
* The `AssignResources` object can be found [here](https://developer.skao.int/projects/ska-telmodel/en/latest/schemas/ska-sdp-assignres.html).
* The `processors` key can be found [here](https://developer.skao.int/projects/ska-sdp-script/en/latest/scripts/vis-receive/configuration.html#processors)

In [ ]:
config = json.dumps(
    {
        "interface": "https://schema.skao.int/ska-sdp-assignres/0.4",
        "resources": {
            "csp_links": [1, 2, 3, 4],
            "receptors": ["C10", "C136", "C1", "C217", "C13", "C42"],
            "receive_nodes": 1,
        },
        "execution_block": {
            "eb_id": EXECUTION_BLOCK_ID,
            "context": {},
            "max_length": 21600.0,
            "channels": [
                {
                    "channels_id": "vis_channels",
                    "spectral_windows": [
                        {
                            "spectral_window_id": "fsp_1_channels",
                            "count": 13824,
                            "start": 0,
                            "stride": 1,
                            "freq_min": 0.35e9,
                            "freq_max": 0.368e9,
                            "link_map": [[0, 0], [200, 1], [744, 2], [944, 3]],
                        }
                    ],
                }
            ],
            "polarisations": [
                {
                    "polarisations_id": "all",
                    "corr_type": ["XX", "XY", "YY", "YX"],
                }
            ],
            "fields": [
                {
                    "field_id": "field_a",
                    "phase_dir": {
                        "ra": [2.711325],
                        "dec": [-0.01328889],
                        "reference_time": "...",
                        "reference_frame": "ICRF3",
                    },
                    "pointing_fqdn": "low-tmc/telstate/0/pointing",
                },
                {
                    "field_id": "field_b",
                    "phase_dir": {
                        "ra": [12.48519],
                        "dec": [2.052388],
                        "reference_time": "...",
                        "reference_frame": "ICRF3",
                    },
                    "pointing_fqdn": "low-tmc/telstate/0/pointing",
                },
            ],
            "beams": [{"beam_id": "vis0", "function": "visibilities"}],
            "scan_types": [
                {
                    "scan_type_id": ".default",
                    "beams": {
                        "vis0": {
                            "polarisations_id": "all",
                            "channels_id": "vis_channels",
                        }
                    },
                },
                {
                    "scan_type_id": "science",
                    "derive_from": ".default",
                    "beams": {"vis0": {"field_id": "field_a"}},
                },
                {
                    "scan_type_id": "calibration",
                    "derive_from": ".default",
                    "beams": {"vis0": {"field_id": "field_b"}},
                },
            ],
        },
        "processing_blocks": [
            {
                "pb_id": PROCESSING_BLOCK_ID_REALTIME,
                "script": {
                    "kind": "realtime",
                    "name": "vis-receive",
                    "version": "4.0.0",
                },
                "parameters": {
                    "use_network_definition": use_network_attachment_definition,
                    "channels_per_port": 6912,
                    "queue_connector_configuration": {
                        "exchanges": [
                            {
                                "dtype": "object",
                                "shape": [],
                                "source": {
                                    "type": "KafkaConsumerSource",
                                    "servers": KAFKA_HOST,
                                    "topic": f"metrics-receive_state-{subarray_id}",
                                    "encoding": "json",
                                },
                                "sink": {
                                    "type": "TangoObjectScatterAttributeSink",
                                    "attributes": [
                                        {
                                            "attribute_name": "receiver_state",
                                            "filter": "type=='visibility_receive'",
                                            "path": "state",
                                            "dtype": "str",
                                            "default_value": "unknown",
                                        },
                                        {
                                            "attribute_name": "last_update",
                                            "filter": "type=='visibility_receive'",
                                            "path": "time",
                                            "dtype": "float",
                                            "default_value": 0.0,
                                        },
                                        {
                                            "attribute_name": "processing_block_id",
                                            "filter": "type=='visibility_receive'",
                                            "path": "processing_block_id",
                                            "dtype": "str",
                                            "default_value": "",
                                        },
                                        {
                                            "attribute_name": "execution_block_id",
                                            "filter": "type=='visibility_receive'",
                                            "path": "execution_block_id",
                                            "dtype": "str",
                                            "default_value": "",
                                        },
                                        {
                                            "attribute_name": "subarray_id",
                                            "filter": "type=='visibility_receive'",
                                            "path": "subarray_id",
                                            "dtype": "str",
                                            "default_value": "-1",
                                        },
                                        {
                                            "attribute_name": "scan_id",
                                            "filter": "type=='visibility_receive'",
                                            "path": "scan_id",
                                            "dtype": "int",
                                            "default_value": 0,
                                        },
                                        {
                                            "attribute_name": "payloads_received",
                                            "filter": "type=='visibility_receive'",
                                            "path": "payloads_received",
                                            "dtype": "int",
                                            "default_value": 0,
                                        },
                                        {
                                            "attribute_name": "time_slices_received",
                                            "filter": "type=='visibility_receive'",
                                            "path": "time_slices",
                                            "dtype": "int",
                                            "default_value": 0,
                                        },
                                        {
                                            "attribute_name": "time_since_last_payload",
                                            "filter": "type=='visibility_receive'",
                                            "path": "time_since_last_payload",
                                            "dtype": "float",
                                            "default_value": 0.0,
                                        },
                                    ],
                                },
                            }
                        ]
                    },
                    "transport_protocol": "tcp",
                    "extra_helm_values": {
                        "receiver": {
                            "options": {
                                "reception": {
                                    "reset_time_indexing_after_each_scan": True,
                                    "stats_receiver_kafka_config": f"{KAFKA_HOST}:json_workflow_state",
                                },
                                "telescope_model": {
                                    "telmodel_key": "instrument/ska1_low/layout/low-layout.json"
                                },
                            }
                        }
                    },
                    "processors": {
                        "signal-display-metrics-amplitude": {},
                        "signal-display-metrics-basic": {},
                        "signal-display-metrics-phase": {},
                        "mswriter": {},
                    },
                    "pod_settings": [
                        {"securityContext": {"runAsUser": 0, "fsGroup": 0}}
                    ],
                },
            }
        ],
    }
)
d.AssignResources(config)

wait_for_state(d, 2)  # IDLE

# 7 Configure the system

In [ ]:
d.Configure(
    '{"interface": "https://schema.skao.int/ska-sdp-configure/0.4", "scan_type": "science"}'
)

wait_for_state(d, 4)  # READY

d.Scan(
    '{"interface": "https://schema.skao.int/ska-sdp-scan/0.4", "scan_id": 1}'
)

wait_for_state(d, 5)  # SCANNING

# 8 Run a Scan

This is the section that runs the actual Scan. And if desired can be run multiple times (as long as the SubArray is still in scanning state)

Refer to the [CBF packetise](https://developer.skao.int/projects/ska-sdp-cbf-emulator/en/latest/api.html#ska_sdp_cbf_emulator.packetiser.packetise) documentation for sending an MS file. And refer to the [Transmitters Config](https://developer.skao.int/projects/ska-sdp-cbf-emulator/en/latest/api.html#ska-sdp-cbf-emulator-transmitters-module) for setting up the config options.

If you are running with `use_network_attachment_definition=True`, you will need to retrieve the internal IP address of the vis-receive pod. You can retrieve the IP address of the newest vis-receive pod with the following command:

In [ ]:
print(
    "kubectl get pods -l app.kubernetes.io/name=vis-receive --namespace %s-sdp -o jsonpath='{.items[0].status.podIP}'"
    % namespace
)

In [ ]:
# Get Receive Addresses
receiveAddresses = json.loads(d.receiveAddresses)
if use_network_attachment_definition:
    vis_receive_ip = "<retrieve this from vis-receive pod>"
    hosts = [[0, vis_receive_ip]]
    receiveAddresses["calibration"]["vis0"]["host"] = hosts
    receiveAddresses["science"]["vis0"]["host"] = hosts

In [ ]:
# pylint: disable=E1142
# Only use one scan_type_id
scan_type_id = "science"

# Only use the first beam_id
beam_id = list(receiveAddresses[scan_type_id].keys())[0]
ms = MeasurementSetDataSourceConfig(location=MS_INPUT_NAME)
config = SenderConfig(ms=ms)
# config.reader.num_repeats = 1  # default
# config.transmission.method = "spead2_transmitters"  # default
config.time_interval = 0

config.transmission = transmitters.create_config(
    scan_id=1,
    target_host=receiveAddresses[scan_type_id][beam_id]["host"][0][1],
    target_port_start=receiveAddresses[scan_type_id][beam_id]["port"][0][1],
    num_streams=2,
    rate=2_822_400,
    transport_protocol="tcp",
    telescope="mid",
    channels_per_stream=6912,
)

print("Total packets sent:", await packetise(config))

# 9 QA Data

Besides the aforementioned dashboards, there is also data available in the Queue Connector Tango device for in progress data:

In [ ]:
queue_connector = DeviceProxy(f"mid-sdp/queueconnector/{subarray_id}")

print(f"Current Receiver State: {queue_connector.receiver_state}")
print(f"Last Update Time: {queue_connector.last_update}")
print(f"Current Processing Block ID: {queue_connector.processing_block_id}")
print(f"Current Execution Block ID: {queue_connector.execution_block_id}")
print(f"Current SubArray ID: {queue_connector.subarray_id}")
print(f"Current Scan ID: {queue_connector.scan_id}")
print(
    f"Current amount of payloads received: {queue_connector.payloads_received}"
)
print(
    f"Current amount of time slices received: {queue_connector.time_slices_received}"
)
print(
    f"Current time since last payloads: {queue_connector.time_since_last_payload}"
)

# 10 Cleanup

This section is optional but only if you plan on deleting the entire deploy. If you plan to rerun the deploy at any point please also run the section.

In [ ]:
print(f"State: {d.obsState.name}")
d.EndScan()
print(f"State: {d.obsState.name}")
d.End()
print(f"State: {d.obsState.name}")
d.ReleaseAllResources()
print(f"State: {d.obsState.name}")

# 11 Get Data from Data Product Dashboard

Documentation for the Data Product Dashboard can be found [here](https://developer.skao.int/projects/ska-dataproduct-dashboard/en/latest/).

The Data Product Dashboard is deployed as a service in available clusters. It should be accessible at: `https://{cluster_domain}/{dpd_namespaces}/dashboard/`, and the backend API should be accessible at: `https://{cluster_domain}/{dpd_namespaces}/api/`

Links to deployed Dashboards (assuming you are using the correct namespace):

* [Mid ITF Cluster](https://k8s.miditf.internal.skao.int/ska-dpd/dashboard/)